In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from amftrack.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests


In [3]:
inst_25 = [(9,0,11),(9,13,35),(3,0,19),(37,0,8),(11,0,30),(19,0,25),(13,0,25),(39,0,18),(35,0,15),(29,0,20)]
inst_bait = [(10,0,10), (14,0,11), (33,0,26),(4,2,18),(4,20,30),(39,117,137),(12,5,21),(28,0,14),(32,5,14),(32,15,44),(36,0,9),(40,0,14),(2,1,15),(5,160,168),(11,158,164),(13,116,131)]
inst_30 = []
inst_25late = [(40,153,153+37)]
treatments = {'25*' : inst_25late,'25' : inst_25,'baits' : inst_bait, '30' : inst_30}
plate_number = {(9,0,11) : 296, (9,13,35) : 296, (3,0,19) : 340, (37,0,8) : 269,(11,0,30) : 314, (19,0,25) : 344, (13,0,25) : 298, (39,0,18) : 297, (35,0,15) : 351,(10,0,10) : 395,(14,0,11) : 399, (33,0,26) : 420, (4,2,18) : 423, (4,20,30) : 423,(8,0,17): 434 ,(8,20,30) : 434,(39,117,137) : 433, (12,5,21) : 436, (28,0,14): 405,(32,5,45):409,(36,0,9) : 419,(40,0,14) : 425,(2,1,15):435,(2,16,35):435,(5,160,168):382,(11,158,164) : 416,(13,116,131) : 424, (29,0,20) : 373,(32,15,44):409, (32,5,14) : 409, (40,153,153+37) : 69}

In [4]:
results={}
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        results[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/maxgrowth_{inst}.pick', "rb"))

In [5]:
column_names = ["plate","inst", "treatment", "max_speed", "total_growth","length"]
infos = pd.DataFrame(columns=column_names)
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        max_growths,total_growths ,lengths = results[inst]
        print(inst,len(max_growths))
        for i,max_growth in enumerate(max_growths):
            new_line = pd.DataFrame(
                {   "plate": [plate_number[inst]],
                    "inst": [inst],
                    "treatment": [treatment],
                    "max_speed": [max_growth],
                    "total_growth": [total_growths[i]],
                    "length": [lengths[i]],
                }
            )  # index 0 for
            # mothers need to be modified to resolve multi mother issue
            infos = infos.append(new_line, ignore_index=True)

2063
8610
2330
614
1867
3097
5060
510
4037
389
946
5718
542
4107
1543
892
11711
2909
2072
2689
2017
1125
903
3904
7485
1550
3110


In [6]:
baits = infos.loc[infos['treatment']=='baits']
control = infos.loc[infos['treatment']=='25']


In [7]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.scatter(np.log10(infos['total_growth']),infos['max_speed'])
ax.set_ylim(0,400)
ax.set_xlabel('log(length) (log(um))')
ax.set_ylabel('max_speed (um.h-1)')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Text(0, 0.5, 'max_speed (um.h-1)')

In [20]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.scatter(np.log10(baits['total_growth']),baits['max_speed'])
ax.set_ylim(0,400)
ax.set_xlabel('log(length) (log(um))')
ax.set_ylabel('max_speed (um.h-1)')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Text(0, 0.5, 'max_speed (um.h-1)')

In [18]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.hist(baits.loc[(baits['max_speed']>100) *(baits['max_speed']<400)]['max_speed'],50)
ax.set_xlim(100,350)
ax.set_xlabel('max_speed (um.h-1)')
ax.set_ylabel('Quantities')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


Text(0, 0.5, 'Quantities')

In [32]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.hist(infos.loc[infos['total_growth']>2000]['total_growth'],50)
ax.set_xlim(0,15000)
ax.set_xlabel('max_speed (um.h-1)')
ax.set_ylabel('Quantities')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Quantities')

In [23]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.hist(control.loc[(control['max_speed']>100) *(control['max_speed']<400)]['max_speed'],50)
ax.set_xlim(100,350)
ax.set_xlabel('max_speed (um.h-1)')
ax.set_ylabel('Quantities')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


Text(0, 0.5, 'Quantities')

In [19]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.hist(infos.loc[(infos['max_speed']>100) *(infos['max_speed']<400)]['max_speed'],50)
ax.set_xlim(100,350)
ax.set_xlabel('max_speed (um.h-1)')
ax.set_ylabel('Quantities')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


Text(0, 0.5, 'Quantities')

In [26]:
RH =infos.loc[(a*infos["length"]+b*infos["max_speed"]>=thresh)&( infos["max_speed"]>=50)&( infos["max_speed"]<500)]

In [5]:
RH

,plate,inst,treatment,max_speed,total_growth,length
10,298,"(13, 0, 19)",25,117.823373,1366.254471,2337.260703
14,298,"(13, 0, 19)",25,182.703809,730.815235,945.888174
27,298,"(13, 0, 19)",25,166.075721,2908.700872,12499.519690
32,298,"(13, 0, 19)",25,99.767964,529.412469,2373.326777
50,298,"(13, 0, 19)",25,101.647627,1023.253658,4509.728855
...,...,...,...,...,...,...
54658,60,"(24, 105, 110)",30,152.669418,687.012381,1135.727461
54661,60,"(24, 105, 110)",30,162.845092,732.802914,1823.542079
54689,60,"(24, 105, 110)",30,169.059829,760.769230,6470.347839
54690,60,"(24, 105, 110)",30,335.168603,1508.258713,5484.742405


In [6]:
infos

,plate,inst,treatment,max_speed,total_growth,length
0,298,"(13, 0, 19)",25,4.547102,72.594697,5892.872576
1,298,"(13, 0, 19)",25,0.000000,0.000000,35.539011
2,298,"(13, 0, 19)",25,2.439518,37.196121,399.168339
3,298,"(13, 0, 19)",25,0.000000,0.000000,0.000000
4,298,"(13, 0, 19)",25,4.247320,61.123850,1135.659767
...,...,...,...,...,...,...
54698,60,"(24, 105, 110)",30,279.701434,1258.656452,2497.702919
54699,60,"(24, 105, 110)",30,0.000000,0.000000,53.647071
54700,60,"(24, 105, 110)",30,0.000000,0.000000,8.625000
54701,60,"(24, 105, 110)",30,637.192769,2867.367463,2867.367463


In [28]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = RH.boxplot(column = ["max_speed"],by="plate",figsize =(9,8),ax =ax,patch_artist=True)
colors = ['pink']*3+['lightblue']*3 +['lightgreen']*3
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Plate number')
ax.set_ylabel(r'Maximum speed ($\mu m.h^{-1}$)')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = RH .boxplot(column = ["max_speed"],by="treatment",figsize =(9,8),ax =ax,patch_artist=True)
colors = ['lightblue'] + ['pink']+['lightgreen']
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Treatment')
ax.set_ylabel(r'Maximum speed ($\mu m.h^{-1}$)')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
np.mean(RH.loc[(infos["treatment"] == "25") & (infos["plate"]!=298)])

plate             84.447154
treatment               inf
max_speed        229.348560
total_growth    1786.923006
length          2977.322864
dtype: float64

In [12]:
np.mean(RH.loc[(infos["treatment"] == "25")])

plate            133.831250
treatment               inf
max_speed        208.673151
total_growth    1896.157875
length          3239.990474
dtype: float64

In [16]:
(298.15+5)/298.15*np.mean(RH.loc[(infos["treatment"] == "25") & (infos["plate"]!=298)])

plate             85.863340
treatment               inf
max_speed        233.194754
total_growth    1816.889851
length          3027.252814
dtype: float64

In [17]:
np.mean(RH.loc[infos["treatment"] == "30"])

plate             47.188329
treatment               inf
max_speed        235.340968
total_growth    1542.979842
length          2817.768091
dtype: float64

In [17]:
ztest ,pval1 = stests.ztest(RH.loc[(infos["treatment"] == "25") & (infos["plate"]!=298)]["max_speed"],RH.loc[infos["treatment"] == "30"]["max_speed"], value=0,alternative='two-sided')
pval1

0.1897215255766116

In [2]:
directory = "/projects/0/einf914/data/"
RH, BAS, max_growths, total_growths, lengths, branch_frequ,select_hyph = get_rh_bas(get_exp((40,0,37),directory),criter)

begin = 2020-11-19 20:21:00 
  end = 2020-11-25 18:06:00


In [3]:
exp = get_exp((40,0,37),directory)

begin = 2020-11-19 20:21:00 
  end = 2020-11-25 18:06:00


In [3]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel(r'maximum length ($\mu m$)')
ax.set_ylabel(r'maximum speed ($\mu m.hour^{-1})$')
ax.set_xlim((0,10000))
ax.set_ylim((0,450))
ax.scatter(total_growths,max_growths,s=5, c='r')
# ax.scatter(lengths,max_growths,s=5, c='b')
ax.plot(range(10000),-a/b*np.array(range(10000))+thresh/b)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [123]:
hyph = choice(BAS)
hyph.ts,hyph.mother,[h.ts for h in hyph.mother]

([13, 14, 15, 16],
 [Hyphae(780,2938), Hyphae(790,781)],
 [[15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
  [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]])

In [124]:
hyph.end.ts()

[13, 14, 15, 16]

In [125]:
hyph.root.ts()

[13, 14, 15, 16]

In [126]:
hyph.end.show_source_image(15,16)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [128]:
plt.close('all')
exp.plot([15,16],[[hyph.end.label,hyph.root.label,780,2938,790,781]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [109]:
hyph.end

Node(5188)

In [67]:
hyph_test = Hyphae(hyph.end)

In [68]:
root,edge,nodes = hyph_test.get_edges(14,200)

(Node(27), Node(540))
(Node(540), Node(702)) 0.42984356469978613
(Node(540), Node(699)) 0.9824784805762508
(Node(540), Node(27)) -1.0
-0.2540324794911799 (Node(702), Node(540))
-1.0 (Node(699), Node(540))
(Node(540), Node(699))
(Node(699), Node(702)) -0.6829248899689792
(Node(699), Node(540)) -1.0
(Node(699), Node(975)) 0.9998709667867004
0.6711022538324886 (Node(702), Node(699))
-1.0 (Node(975), Node(699))
(Node(699), Node(975))
(Node(975), Node(699)) -1.0
(Node(975), Node(1553)) 0.5446654754608982
(Node(975), Node(1219)) 0.9748633281968202
-0.3441193134538916 (Node(1553), Node(975))
-1.0 (Node(1219), Node(975))
(Node(975), Node(1219))
(Node(1219), Node(975)) -1.0
(Node(1219), Node(1534)) 0.9924433926887687
(Node(1219), Node(1224)) 0.029440248780633255
-1.0 (Node(1534), Node(1219))
0.09343238370551364 (Node(1224), Node(1219))
(Node(1219), Node(1534))
(Node(1534), Node(1219)) -1.0
(Node(1534), Node(1531)) 0.7407862905431174
(Node(1534), Node(1840)) 0.9651140194598891
-0.539061371694873

In [69]:
root

Node(226)

In [74]:
roots=[]
for t in hyph.end.ts():
    #             print(t,tip)
    if hyph.end.degree(t) == 1:
        root, edges, nodes = hyph.get_edges(t, 200)
        roots.append(root)
occurence_count = Counter(roots)

In [75]:
occurence_count

Counter({Node(96): 6,
         Node(53): 2,
         Node(468): 1,
         Node(1174): 2,
         Node(1750): 1,
         Node(443): 1,
         Node(2240): 1})

In [14]:
class Hyphae:
    def __init__(self, tip):
        self.experiment = tip.experiment
        self.ts = tip.ts()
        self.end = tip
        self.root = None
        self.mother = None

    def __eq__(self, other):
        return self.end.label == other.end.label

    def __repr__(self):
        return f"Hyphae({self.end},{self.root})"

    def __str__(self):
        return str((self.end, self.root))

    def __hash__(self):
        return self.end.label

    def get_root(self, t, length=100):
        return(None)
    
    def get_edges(self, t, length=100):
        first_neighbour = self.end.neighbours(t)[0]
        last_node = self.end
        current_node = first_neighbour
        current_edge = Edge(last_node, current_node, self.experiment)
        moving_on_hyphae = True
        edges = [current_edge]
        nodes = [last_node, current_node]
        i = 0
        while moving_on_hyphae:
            print(current_edge)
            i += 1
            if i >= 100:
                print(t, self.end, current_node)
            if i >= 110:
                break
            #                 print ('moving',current_node)
            if current_node.degree(t) < 2:
                #                     print(current_node.degree(t),current_node)
                moving_on_hyphae = False
            else:
                maxi = -np.inf
                orientation = current_edge.orientation_end(t, length)
                for neighbours_t in current_node.neighbours(t):
                    #                     print (neighbours_t)
                    candidate_edge = Edge(current_node, neighbours_t, self.experiment)
                    orientation_candidate = candidate_edge.orientation_begin(t, length)
                    angle = np.cos(
                        (orientation - orientation_candidate) / 360 * 2 * np.pi
                    )
                    print(candidate_edge,angle)
                    if angle > maxi:
                        maxi = angle
                        next_node_candidate = neighbours_t
                #                     print(maxi,next_node_candidate)
                candidate_edge = Edge(
                    current_node, next_node_candidate, self.experiment
                )
                orientation_candidate = candidate_edge.orientation_begin(t, length)
                maxi_compet = -np.inf
                #                     print('compet')
                for neighbours_t in current_node.neighbours(t):
                    if neighbours_t != last_node:
                        competitor_edge = Edge(
                            neighbours_t, current_node, self.experiment
                        )
                        orientation_competitor = competitor_edge.orientation_end(
                            t, length
                        )
                        angle = np.cos(
                            (orientation_competitor - orientation_candidate)
                            / 360
                            * 2
                            * np.pi
                        )
                        print(angle,competitor_edge)
                        if angle > maxi_compet:
                            maxi_compet = angle
                            competitor = neighbours_t
                #                             print(neighbours_t,angle)
                #                     print(maxi_compet,competitor)
                if maxi_compet > maxi:
                    moving_on_hyphae = False
                else:
                    last_node, current_node = current_node, next_node_candidate
                    current_edge = Edge(last_node, current_node, self.experiment)
                    edges.append(current_edge)
                    nodes.append(current_node)
        #         while moving:
        #             c= move_hyphae(llast_node,ccurrent_node)
        #             edges += c[0]
        #             nodes += c[1]
        #             competitor = c[2]
        # #             print('moving back', nodes[-1],competitor)
        #             move_backward = move_hyphae(nodes[-1],competitor)
        #             end_node_move_backward = move_backward[1][-1]
        #             if end_node_move_backward in nodes:
        # #                 print('restarting',competitor,nodes[-1])
        #                 llast_node,ccurrent_node = c[2],nodes[-1]
        #             else:
        #                 moving=False
        root = nodes[-1]
        edges = edges
        nodes = nodes
        return (root, edges, nodes)

    def get_nodes_within(self, t):
        nodes = nx.shortest_path(
            self.experiment.nx_graph[t],
            source=self.root.label,
            target=self.end.label,
            weight="weight",
        )
        edges = [
            Edge(
                self.experiment.get_node(nodes[i]),
                self.experiment.get_node(nodes[i + 1]),
                self.experiment,
            )
            for i in range(len(nodes) - 1)
        ]
        return (nodes, edges)

    def get_length_pixel(self, t):
        nodes, edges = self.get_nodes_within(t)
        length = 0
        for edge in edges:
            length += len(edge.pixel_list(t))
        return length

    def get_length_um(self, t):
        pixel_conversion_factor = 1.725
        nodes, edges = self.get_nodes_within(t)
        length = 0
        for edge in edges:
            length_edge = 0
            pixels = edge.pixel_list(t)
            for i in range(len(pixels) // 10 + 1):
                if i * 10 <= len(pixels) - 1:
                    length_edge += np.linalg.norm(
                        np.array(pixels[i * 10])
                        - np.array(pixels[min((i + 1) * 10, len(pixels) - 1)])
                    )
            #         length_edge+=np.linalg.norm(np.array(pixels[len(pixels)//10-1*10-1])-np.array(pixels[-1]))
            length += length_edge
        return length * pixel_conversion_factor

    def get_mother(self):
        candidate_mother = []
        for hyphae in self.experiment.hyphaes:
            for t in hyphae.ts:
                if self.root in hyphae.get_nodes_within(t):
                    candidate_mother.append(hyphae)
        self.mother = candidate_mother

    def update_ts(self):
        self.ts = sorted(set(self.end.ts()).intersection(set(self.root.ts())))

In [10]:
total_growth_dic ={}
for bas in BAS:
    hyph = bas
    speeds = [c[2] for c in select_hyph[hyph]]
    ts = [c[0] for c in select_hyph[hyph]]
    tp1s = [c[1] for c in select_hyph[hyph]]
    total_growth = np.sum([speed * get_time(hyph.experiment,ts[i],tp1s[i]) for i,speed in enumerate(speeds)])
    total_growth_dic[bas] = total_growth

In [11]:
BAS_interest = [bas for bas in BAS if total_growth_dic[bas]>=1000]

In [38]:
len(BAS_interest)

13

In [70]:
total_growth_dic[hyph]

1033.459273666884

In [66]:
hyph = choice(BAS_interest)
hyph.ts

[29, 30, 31, 32, 33, 34, 35, 36, 37]

In [69]:
plt.close('all')
t0 = select_hyph[hyph][0][0]
times=[get_time(rh.experiment,t0,c[0]) for c in select_hyph[hyph]]
speeds = [c[2] for c in select_hyph[hyph]]
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time (hour)')
ax.set_ylabel('speed (um/hour)')
ax.plot(times,speeds)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
plt.close('all')
rh.experiment.plot(hyph.ts,[[hyph.end.label,hyph.root.label]]*len(hyph.ts))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
1300/24

54.166666666666664

In [68]:
plt.close('all')
hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])
hyph.root.show_source_image(hyph.ts[-1],hyph.ts[-1])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
rh.experiment.plot_raw(hyph.ts[-1])

NameError: name 'exp' is not defined

In [21]:
window = 800
hyph_infos = {}
skeletons = [sparse.csr_matrix(skel) for skel in rh.experiment.skeletons]
for j,hyph in enumerate(RH):
    hyph_infos[hyph] = []
    for i,t in enumerate(hyph.ts[:-1]):
        tp1=hyph.ts[i+1]
        pixels,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
        speed = np.sum([get_length_um(seg) for seg in pixels])/get_time(rh.experiment,t,tp1)
        pos = hyph.end.pos(t)
        x,y = pos[0],pos[1]
        skeleton=skeletons[t][x-window:x+window,y-window:y+window]
        density = skeleton.count_nonzero()/window
        hyph_infos[hyph].append((t,hyph.ts[i+1],speed,pixels,density))
speeds = []
densities = []
for hyph in RH:
    infos = hyph_infos[hyph]
    for info in infos:
        speed = info[2]
        density = info[4]
        if speed>=100 and hyph.end.degree(info[1])<3 and density>0:
            speeds.append(speed)
            densities.append(density/1.725)
slope, intercept, r_value, p_value, std_err = stats.linregress(densities,speeds)

In [49]:
zipped_list = zip(densities,speeds)
sorted_zip = sorted(zipped_list)
densities_sort,speed_sort = zip(*sorted_zip)
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel(r'density ($\mu m .\mu m^{-1})$')
ax.set_ylabel('speed ($\mu m.hour^{-1})$')
xlim=8
ax.set_xlim((0,xlim))
ax.set_ylim((0,450))
# N=len(densities)
N=30
ax.scatter(densities_sort,speed_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(densities_sort, size=N),uniform_filter1d(speed_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)

ax.legend()
r_value**2,p_value, slope

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.13304401016453313, 1.1901009796021612e-17, -14.397045471705182)

In [53]:
selected =[]
for hyph in RH:
    infos = hyph_infos[hyph]
    for info in infos:
        speed = info[2]
        density = info[4]
        t = info[0]
        tp1 = info[1]
        if speed<=400 and density>=3 and speed>=100 and hyph.end.degree(info[1])<3:
            selected.append((hyph,t,tp1,speed,density))

In [84]:
# hyph,t,tp1,speed,density = choice(selected)
label = 27

hyph,t,tp1,speed,density = [example for example in [hyph for hyph in selected if hyph[0].end.label==label]][-1]


In [70]:
hyph, t,tp1,speed,density

(Hyphae(1049,246), 11, 12, 148.22632862084015, 4.04125)

In [85]:
hyph.end.show_source_image(t,tp1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.991962388624081

In [46]:
np.linalg.norm(hyph.end.pos(t)-hyph.end.pos(tp1))

424.6339600173307

In [89]:
window = 800
hyph_infos = {}
skeletons = [sparse.csr_matrix(skel) for skel in rh.experiment.skeletons]
for j,hyph in enumerate(RH):
    hyph_infos[hyph] = []
    for i,t in enumerate(hyph.ts[:-1]):
        tp1=hyph.ts[i+1]
        pixels,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
        speed = np.sum([get_length_um(seg) for seg in pixels])/get_time(rh.experiment,t,tp1)
        pos = hyph.end.pos(t)
        x,y = pos[0],pos[1]
        if speed>0:
            straightness = np.linalg.norm(hyph.end.pos(t)-hyph.end.pos(tp1))*1.725/(speed*get_time(hyph.experiment,t,tp1))
            hyph_infos[hyph].append((t,hyph.ts[i+1],speed,pixels,straightness))

In [107]:
selected =[]
for hyph in RH:
    infos = hyph_infos[hyph]
    for info in infos:
        speed = info[2]
        straightness = info[4]
        t = info[0]
        tp1 = info[1]
        if speed<=400 and straightness<=0.97 and speed>=100 and hyph.end.degree(info[1])<3:
            selected.append((hyph,t,tp1,speed,straightness))

In [101]:
len(selected)

54

In [126]:
plt.close('all')
hyph,t,tp1,speed,straightness = choice(selected)
speed,straightness

(135.96895995284052, 0.968780747471544)

In [127]:
hyph.end.show_source_image(t,tp1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
from scipy.ndimage.filters import uniform_filter1d
uniform_filter1d(speeds, size=10).shape,len(speeds)

((515,), 515)

In [14]:
np.convolve(speeds, np.ones(N)/N, mode='valid')

array([222.63508958])

In [9]:
1400/1.725

811.5942028985506

In [5]:
window = 2000
selected = []
skeletons = [sparse.csr_matrix(skel) for skel in rh.experiment.skeletons]
for j,hyph in enumerate(RH):
    hyph_infos[hyph] = []
    for i,t in enumerate(hyph.ts[:-1]):
        tp1=hyph.ts[i+1]
        pixels,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
        speed = np.sum([get_length_um(seg) for seg in pixels])/get_time(rh.experiment,t,tp1)
        pos = hyph.end.pos(t)
        x,y = pos[0],pos[1]
        skeleton=skeletons[t][x-window:x+window,y-window:y+window]
        density = skeleton.count_nonzero()/window
        if speed<=150 and speed>=100 and hyph.end.degree(tp1)<3:
            selected.append((hyph,t,hyph.ts[i+1],speed,pixels,density))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-9921a2213305>", line 5, in <module>
    hyph_infos[hyph] = []
NameError: name 'hyph_infos' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1170, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  F

TypeError: object of type 'NoneType' has no len()

In [69]:
hyph,t,tp1,speed,pixels,density = choice(selected)
print(speed,density)

105.9093037130196 35.0735


In [70]:
plt.close('all')
hyph.end.show_source_image(t,tp1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
hyph.experiment.plot([t,tp1],[[hyph.end.label]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
t0 = 0
speeds = []
times=[]
for hyph in RH:
    infos = hyph_infos[hyph]
    for info in infos:
        time = get_time(rh.experiment,t0,info[0])
        speed = info[2]
        density = info[4]
        if speed>=100 and hyph.end.degree(info[1])<3 and density>0:
            speeds.append(speed)
            times.append(time)

In [49]:
slope, intercept, r_value, p_value, std_err = stats.linregress(times,speeds)

In [56]:
zipped_list = zip(times,speeds)
sorted_zip = sorted(zipped_list)
times_sort,speed_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time since start of imaging ($hour$)')
ax.set_ylabel('speed ($\mu m.hour^{-1})$')
xlim = int(get_time(rh.experiment,t0,len(rh.experiment.nx_graph)-1))+1
ax.set_xlim((0,xlim))
ax.set_ylim((0,450))

x= range(0,xlim)

N=30
ax.scatter(times_sort,speed_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(times_sort, size=N),uniform_filter1d(speed_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)
ax.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
def estimate_growth_mult(insts,criter):
    max_growths_exp=[]
    for inst in insts:
        max_growth_rh = []
        exp = get_exp(inst)
        RH, BAS, max_growths, total_growths, lengths, branch_frequ, select_hyph = get_rh_bas(exp,criter)
        print(inst,len(RH))
        for rh in RH:
            growths = [c[2] for c in select_hyph[rh]]
            max_growth = np.max(growths)
            max_growth_rh.append(max_growth)
        max_growths_exp.append(max_growth_rh)
    return(max_growths_exp)

In [12]:
inst_25 = [(33,5,25),(39,151,165),(14,0,11)]
max_growths25 = estimate_growth_mult(inst_25,criter)


begin = 2020-09-29 15:17:00 
  end = 2020-10-03 19:34:00
(33, 5, 25) 237
begin = 2020-10-01 15:52:00 
  end = 2020-10-05 11:16:00
(39, 151, 165) 57
begin = 2020-09-22 13:17:00 
  end = 2020-09-24 09:18:00
(14, 0, 11) 32


In [13]:
len(max_growths25)

326

In [15]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.hist(max_growths25,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([27., 37., 66., 46., 56., 47., 23.,  4.,  3.,  1.,  3.,  1.,  2.,
         2.,  0.,  1.,  1.,  3.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.]),
 array([  50.09833821,   93.47335413,  136.84837005,  180.22338598,
         223.5984019 ,  266.97341782,  310.34843375,  353.72344967,
         397.09846559,  440.47348152,  483.84849744,  527.22351336,
         570.59852929,  613.97354521,  657.34856113,  700.72357706,
         744.09859298,  787.4736089 ,  830.84862483,  874.22364075,
         917.59865667,  960.9736726 , 1004.34868852, 1047.72370444,
        1091.09872037, 1134.47373629, 1177.84875221, 1221.22376814,
        1264.59878406, 1307.97379998, 1351.34881591]),
 <a list of 30 Patch objects>)

In [87]:
selected =[]
for hyph in RH:
    infos = hyph_infos[hyph]
    for info in infos:
        speed = info[2]
        density = info[4]
        t = info[0]
        tp1 = info[1]
        if speed<=150 and np.log(density)>=-5 and speed>=10 and hyph.end.degree(info[1])<3:
            selected.append((hyph,t,tp1,speed,density))

In [99]:
hyph,t,tp1,speed,density = choice(selected)
speed,density,np.log(density)

(24.009527606759786, 0.011084, -4.502252651981405)

In [102]:
plt.close('all')
exp.plot([t,tp1],[[hyph.end.label,hyph.root.label]]*2)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [101]:
rh=hyph
plt.close('all')
t0 = select_hyph[rh][0][0]
times=[get_time(exp,t0,c[0]) for c in select_hyph[rh]]
speeds = [c[2] for c in select_hyph[rh]]
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time (hour)')
ax.set_ylabel('speed (um/hour)')
ax.plot(times,speeds)

KeyError: Hyphae(1150,96)

In [98]:
t,tp1,hyph.ts

(33, 34, [33, 34, 35, 36, 37])

In [45]:
hyph = choice(exp40.hyphaes)

In [46]:
t = hyph.ts[-1]
pos = hyph.end.pos(t)

In [47]:
window = 500
x,y = pos[0],pos[1]
skeleton=exp.skeletons[t][x-window:x+window,y-window:y+window]

In [48]:
density = len(skeleton.nonzero()[0])/window
density

5.816

In [23]:
hyph.end.show_source_image(t,t)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.scatter(x,y)
ax.imshow(skeleton.todense())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('maximum length (um)')
ax.set_ylabel('maximum speed (um/hour)')
ax.set_xlim((0,15000))
ax.set_ylim((0,450))
ax.scatter(lengths,max_growths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time (hour)')
ax.set_ylabel('speed (um/hour)')
for i in range(50):
    rh=choice(RH)
    t0 = select_hyph[rh][0][0]
    times=[get_time(exp,t0,c[0]) for c in select_hyph[rh]]
    speeds = [c[2] for c in select_hyph[rh]]
    if np.max(speeds)<=600:
        ax.plot(times,speeds)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
rh = choice(RH)
rh.ts

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [60]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('maximum speed (um/hour)')
ax.set_ylabel('density')
ax.set_xlim((0,450))
ax.hist([s for s in max_growths if s>=20 and s<=600],30,density=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([2.08318210e-02, 1.16792959e-02, 9.04022420e-03, 5.55889563e-03,
        2.80752304e-03, 2.30216890e-03, 1.51606244e-03, 1.01070830e-03,
        1.23531014e-03, 6.17655070e-04, 6.73805531e-04, 5.05354148e-04,
        9.54557835e-04, 8.42256913e-04, 8.42256913e-04, 1.06685876e-03,
        9.54557835e-04, 9.54557835e-04, 7.29955992e-04, 1.12300922e-04,
        5.61504609e-05, 5.61504609e-05, 0.00000000e+00, 0.00000000e+00,
        5.61504609e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        5.61504609e-05, 5.61504609e-05]),
 array([ 20.01257891,  35.51239859,  51.01221828,  66.51203796,
         82.01185765,  97.51167733, 113.01149702, 128.5113167 ,
        144.01113638, 159.51095607, 175.01077575, 190.51059544,
        206.01041512, 221.51023481, 237.01005449, 252.50987418,
        268.00969386, 283.50951355, 299.00933323, 314.50915292,
        330.0089726 , 345.50879229, 361.00861197, 376.50843166,
        392.00825134, 407.50807103, 423.00789071, 438.50771039,
      

In [45]:
plt.close('all')
bas = np.random.choice(RH)
print(bas.ts)
exp.plot([bas.ts[0],bas.ts[-1]],[[bas.end.label,bas.root.label]]*2)

[1, 2, 3, 4, 5]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.close('all')
fig=plt.figure(figsize=(8,8))
# ax = fig.add_subplot(111,projection='3d')
ax = fig.gca(projection='3d')
X,Y,Z=np.log(lengths),np.log(max_growths),np.array([np.log(branch_frequ).tolist(),db.labels_.tolist()])
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
# ax.scatter()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


ValueError: Argument Z must be 2-dimensional.

In [94]:
a = np.log(branch_frequ).tolist()
b = db.labels_.tolist()
len(a)
len(b)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


3248

In [91]:
Z

array([list([-6.602227002083715, -3.598380486698803, -5.991885306651625, -inf, -7.0358492116722235, -6.130039349429718, -6.43043307950541, -5.039829205441329, -5.933537432051992, -3.9848059223036136, -6.658550354152661, -7.574187714752405, -5.576821060176731, -5.285032022024386, -6.853181001249265, -4.743004703817778, -5.100338563059207, -inf, -4.528914222506007, -5.575318840669745, -4.1576488543936785, -4.476117151500033, -4.916364731366313, -6.689533530193552, -4.498804454561235, -6.445395065729935, -5.055052914884203, -6.214649672777758, -5.075445900349605, -4.376057896922178, -5.901766301368058, -5.196843384052745, -6.3861748527966355, -6.26841305845049, -4.897167907626487, -5.244041945774212, -5.3516126869287275, -4.322082548550157, -5.405060410621938, -6.61760596224269, -6.607270848325521, -6.239702702347082, -6.148650440258393, -5.278632876987552, -5.696693708957829, -3.3567416878042033, -5.57131657152519, -4.384951269595142, -6.011540409778838, -4.042058810241346, -6.6224545589

In [96]:
from sklearn.cluster import MiniBatchKMeans,DBSCAN
kmeans = MiniBatchKMeans(n_clusters=2)
X = np.array([np.log(np.array(lengths)+1),np.log(np.array(max_growths)+1),np.log(np.array(branch_frequ)+1)])
kmeans = kmeans.fit(np.transpose(X))

In [98]:
from sklearn.cluster import MiniBatchKMeans,DBSCAN
db = DBSCAN(eps=0.1, min_samples=30)
X = np.array([np.log(np.array(lengths)+1),np.log(np.array(max_growths)+1),np.log(np.array(branch_frequ)+1)])
db = db.fit(np.transpose(X))

In [27]:
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(n_clusters=2)
X = np.array([np.array(lengths),np.array(max_growths),np.array(branch_frequ)])
kmeans = kmeans.fit(np.transpose(X))

In [30]:
from mpl_toolkits.mplot3d import Axes3D

plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111,projection='3d')
ax.scatter(lengths,max_growths,kmeans.labels_)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
BAS = [hyph for i,hyph in enumerate(hyph_l) if kmeans.labels_[i]==0]

In [16]:
ambiguous = [hyph for i,hyph in enumerate(hyph_l) if kmeans.labels_[i]==2]

In [45]:
RH = [hyph for i,hyph in enumerate(hyph_l) if kmeans.labels_[i]==1]

In [61]:
bas_save = bas

In [62]:
plt.close('all')
bas = np.random.choice(exp.hyphaes)
print(bas.ts)
exp.plot([bas.ts[0],bas.ts[-1]],[[bas.end.label,bas.root.label]]*2)

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
from experiment_class_surftest import Edge
class Hyphae:
    def __init__(self, tip):
        self.experiment = tip.experiment
        self.ts = tip.ts()
        self.end = tip
        self.root = None
        self.mother = None

    def __eq__(self, other):
        return self.end.label == other.end.label

    def __repr__(self):
        return f"Hyphae({self.end},{self.root})"

    def __str__(self):
        return str((self.end, self.root))

    def __hash__(self):
        return self.end.label

    def get_edges(self, t, length=150):
        first_neighbour = self.end.neighbours(t)[0]
        last_node = self.end
        current_node = first_neighbour
        current_edge = Edge(last_node, current_node, self.experiment)
        moving_on_hyphae = True
        edges = [current_edge]
        nodes = [last_node, current_node]
        i = 0
        while moving_on_hyphae:
            i += 1
            if i >= 100:
                print(t, self.end, current_node)
            if i >= 110:
                break
            #                 print ('moving',current_node)
            if current_node.degree(t) < 2:
                #                     print(current_node.degree(t),current_node)
                moving_on_hyphae = False
            else:
                mini = np.inf
                orientation = current_edge.orientation_end(t, length)
                for neighbours_t in current_node.neighbours(t):
                    #                     print (neighbours_t)
                    candidate_edge = Edge(current_node, neighbours_t, self.experiment)
                    orientation_candidate = candidate_edge.orientation_begin(t, length)
                    angle = np.cos(
                        (orientation - orientation_candidate) / 360 * 2 * np.pi
                    )
                    angle = np.arccos(angle)
                    score = angle/min(len(candidate_edge.pixel_list(t)),length)
                    if score < mini:
                        mini = angle
                        next_node_candidate = neighbours_t
                #                     print(maxi,next_node_candidate)
                candidate_edge = Edge(
                    current_node, next_node_candidate, self.experiment
                )
                orientation_candidate = candidate_edge.orientation_begin(t, length)
                mini_compet = np.inf
                #                     print('compet')
                for neighbours_t in current_node.neighbours(t):
                    if neighbours_t != last_node:
                        competitor_edge = Edge(
                            neighbours_t, current_node, self.experiment
                        )
                        orientation_competitor = competitor_edge.orientation_end(
                            t, length
                        )
                        angle = np.cos(
                            (orientation_competitor - orientation_candidate)
                            / 360
                            * 2
                            * np.pi
                        )
                        angle = np.arccos(angle)
                        score = angle/min(len(competitor_edge.pixel_list(t)),length)
                        if score < mini_compet:
                            mini_compet = angle
                            competitor = neighbours_t
                #                             print(neighbours_t,angle)
                #                     print(maxi_compet,competitor)
                if mini_compet < mini:
                    moving_on_hyphae = False
                else:
                    last_node, current_node = current_node, next_node_candidate
                    current_edge = Edge(last_node, current_node, self.experiment)
                    edges.append(current_edge)
                    nodes.append(current_node)
        #         while moving:
        #             c= move_hyphae(llast_node,ccurrent_node)
        #             edges += c[0]
        #             nodes += c[1]
        #             competitor = c[2]
        # #             print('moving back', nodes[-1],competitor)
        #             move_backward = move_hyphae(nodes[-1],competitor)
        #             end_node_move_backward = move_backward[1][-1]
        #             if end_node_move_backward in nodes:
        # #                 print('restarting',competitor,nodes[-1])
        #                 llast_node,ccurrent_node = c[2],nodes[-1]
        #             else:
        #                 moving=False
        root = nodes[-1]
        edges = edges
        nodes = nodes
        return (root, edges, nodes)

    def get_nodes_within(self, t):
        nodes = nx.shortest_path(
            self.experiment.nx_graph[t],
            source=self.root.label,
            target=self.end.label,
            weight="weight",
        )
        edges = [
            Edge(
                self.experiment.get_node(nodes[i]),
                self.experiment.get_node(nodes[i + 1]),
                self.experiment,
            )
            for i in range(len(nodes) - 1)
        ]
        return (nodes, edges)

    def get_length_pixel(self, t):
        nodes, edges = self.get_nodes_within(t)
        length = 0
        for edge in edges:
            length += len(edge.pixel_list(t))
        return length

    def get_length_um(self, t):
        pixel_conversion_factor = 1.725
        nodes, edges = self.get_nodes_within(t)
        length = 0
        for edge in edges:
            length_edge = 0
            pixels = edge.pixel_list(t)
            for i in range(len(pixels) // 10 + 1):
                if i * 10 <= len(pixels) - 1:
                    length_edge += np.linalg.norm(
                        np.array(pixels[i * 10])
                        - np.array(pixels[min((i + 1) * 10, len(pixels) - 1)])
                    )
            #         length_edge+=np.linalg.norm(np.array(pixels[len(pixels)//10-1*10-1])-np.array(pixels[-1]))
            length += length_edge
        return length * pixel_conversion_factor

    def get_mother(self):
        candidate_mother = []
        for hyphae in self.experiment.hyphaes:
            for t in hyphae.ts:
                if self.root in hyphae.get_nodes_within(t):
                    candidate_mother.append(hyphae)
        self.mother = candidate_mother

    def update_ts(self):
        self.ts = sorted(set(self.end.ts()).intersection(set(self.root.ts())))

In [19]:
Hyphae(bas.end).get_edges(19)

(Node(3933), [Edge(3927,3933)], [Node(3927), Node(3933)])

In [21]:
bas.end.degree(18)

1

In [48]:
RH_fast = []
RH_slow = []
for hyph in RH:
    growths = [c[2] for c in select_hyph[hyph]]
    speed = np.max(growths)
    if speed >=130:
        RH_fast.append(hyph)
    else:
        RH_slow.append(hyph)

In [109]:
plt.close('all')
speeds=[]
for hyph in exp.hyphaes:
    growths = [c[2] for c in select_hyph[hyph]]
    if len(growths)>0:
        speed = np.max(growths)
        if speed>=50:
            speeds.append(speed)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.hist(speeds,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([205.,  77.,  11.,   8.,   4.,   2.,   2.,   1.,   0.,   0.,   1.,
          1.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,
          0.,   1.,   0.,   0.,   0.,   0.,   0.,   1.]),
 array([  50.        ,   95.06666667,  140.13333333,  185.2       ,
         230.26666667,  275.33333333,  320.4       ,  365.46666667,
         410.53333333,  455.6       ,  500.66666667,  545.73333333,
         590.8       ,  635.86666667,  680.93333333,  726.        ,
         771.06666667,  816.13333333,  861.2       ,  906.26666667,
         951.33333333,  996.4       , 1041.46666667, 1086.53333333,
        1131.6       , 1176.66666667, 1221.73333333, 1266.8       ,
        1311.86666667, 1356.93333333, 1402.        ]),
 <a list of 30 Patch objects>)

In [41]:
overgrowing = [c for lis in select_hyph.values() for c in lis if c[2]>=20 and c[2]<1500 and c[0]<=20 and c[0]>13]
selected_growth = [c[2] for c in overgrowing]
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.hist(selected_growth,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([240.,  37.,  32.,  26.,  19.,  25.,  19.,  36.,  34.,  32.,  12.,
          1.,   2.,   0.,   0.,   1.,   0.,   3.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.]),
 array([  20.        ,   58.43333333,   96.86666667,  135.3       ,
         173.73333333,  212.16666667,  250.6       ,  289.03333333,
         327.46666667,  365.9       ,  404.33333333,  442.76666667,
         481.2       ,  519.63333333,  558.06666667,  596.5       ,
         634.93333333,  673.36666667,  711.8       ,  750.23333333,
         788.66666667,  827.1       ,  865.53333333,  903.96666667,
         942.4       ,  980.83333333, 1019.26666667, 1057.7       ,
        1096.13333333, 1134.56666667, 1173.        ]),
 <a list of 30 Patch objects>)

In [33]:
overgrowing = [c for c in select_hyph if c[3]>=20]
len_growth = [(c[3],c[0].get_length_um(c[0].ts[-1]),c[0]) for c in overgrowing]

In [ ]:
for c in 

In [34]:
growth = [c[0] for c in len_growth if c[0]<=2000]
length = [c[1] for c in len_growth if c[0]<=2000]
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(growth,np.log(length))

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [101]:
overgrowing = [c for c in select_hyph if c[3]>=20]
selected_growth = [c[3] for c in overgrowing]
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.hist(selected_growth,30)

TypeError: 'Hyphae' object is not subscriptable

In [10]:
plt.close('all')
overgrowing = [c for c in select_hyph if c[3]>=20]
selected_growth = [c[3] for c in overgrowing]
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.hist(selected_growth,30)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([353., 167., 101.,  59.,  31.,  28.,  30.,  31.,  14.,  25.,  26.,
         29.,  43.,  40.,  42.,  33.,   1.,   0.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.]),
 array([  20. ,   65.8,  111.6,  157.4,  203.2,  249. ,  294.8,  340.6,
         386.4,  432.2,  478. ,  523.8,  569.6,  615.4,  661.2,  707. ,
         752.8,  798.6,  844.4,  890.2,  936. ,  981.8, 1027.6, 1073.4,
        1119.2, 1165. , 1210.8, 1256.6, 1302.4, 1348.2, 1394. ]),
 <a list of 30 Patch objects>)

In [71]:
hyph,t,tp1,le = choice(overgrowing)
print(le)

45


In [72]:
plt.close('all')
exp.plot([t,tp1],[[hyph.end.label,hyph.root.label]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
hyph.end.show_source_image(t,tp1)

OSError: The file /scratch/shared/AMF914/Fiji.app/20201121_0421_Plate40/Analysis/skeleton_realigned.mat does not exist.

In [5]:
growth_length = [np.sum([len(seg) for seg in segs]) for segs in growth]

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([353., 167., 101.,  59.,  31.,  28.,  30.,  31.,  14.,  25.,  26.,
         29.,  43.,  40.,  42.,  33.,   1.,   0.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.]),
 array([  20. ,   65.8,  111.6,  157.4,  203.2,  249. ,  294.8,  340.6,
         386.4,  432.2,  478. ,  523.8,  569.6,  615.4,  661.2,  707. ,
         752.8,  798.6,  844.4,  890.2,  936. ,  981.8, 1027.6, 1073.4,
        1119.2, 1165. , 1210.8, 1256.6, 1302.4, 1348.2, 1394. ]),
 <a list of 30 Patch objects>)

In [15]:
np.mean(branch_frequ)

0.021061224646895957

In [5]:
plt.close('all')
expt= exp40
t = 0
nodes = np.random.choice(expt.nx_graph[t].nodes, 100)
# exp.plot([8,13,14],[[794]]*3)
expt.plot([t,t+1,t+2],[nodes]*3)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
sys.path.append( '/home/cbisot/pycode/MscThesis/amftrack/pipeline/functions')
from amftrack.notebooks.analysis.util import *
from amftrack.pipeline.paths.directory import path_code, directory_scratch, directory_project
from amftrack.notebooks.analysis.data_info import *
path = directory_project
for treatment in ['baits']:
    insts = treatments[treatment]
    for inst in insts[1:]:
        print(inst)
        exp = get_exp(inst, path)
        break

(14, 0, 11)
begin = 2020-09-22 13:17:00 
  end = 2020-09-24 09:18:00


In [18]:
hyph = choice(exp.hyphaes)

In [19]:
hyph.get_nodes_within(hyph.ts[-1])[1][0].width(-1)

KeyError: 'width'